In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.19 pyspark-shell'

from time import sleep

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [2]:
spark = SparkSession. \
    builder. \
    appName("Spark DSL"). \
    master("local[4]"). \
    config("spark.sql.legacy.timeParserPolicy", "LEGACY"). \
    config("spark.sql.autoBroadcastJoinThreshold", -1). \
    getOrCreate()

In [3]:
movies_df = spark.read.json("data/movies")

# Aggregations

## Counting

In [4]:
movies_df.printSchema()

In [5]:
# NULL значения не участвуют в подсчете
all_movies_count_df = movies_df.selectExpr("count(Major_Genre)")
all_movies_count_df.show()

In [6]:
all_movies_count_df.explain()

In [7]:
# NULL значения не участвуют в подсчете
genres_count_df = movies_df.select(count(col("Major_Genre")))
genres_count_df.show()

In [8]:
genres_count_df.explain()

In [9]:
# NULL значения включены в подсчет строк
genres_count_df = movies_df.select(count(col("*")).alias("Общее колличество строк"))
genres_count_df.show()

In [10]:
genres_count_df.explain()

In [11]:
# NULL значения включены в подсчет строк
genres_count_df_v2 = movies_df.selectExpr('count(*) as Total_Rows')
genres_count_df_v2.show()

In [12]:
# NULL значения включены в подсчет строк, `.count()` -- Action
genres_count_number = movies_df.select("Major_Genre").count()
print("Общее колличество строк:", genres_count_number)

In [13]:
# Подсчет уникальных значений
unique_genres_df = movies_df.select(F.countDistinct(F.col("Major_Genre")).alias("Уникальное число жанров"))
unique_genres_df.show()

In [14]:
unique_genres_df_v2 = movies_df.selectExpr("count(DISTINCT Major_Genre) as Number_Of_Unique_Genres")
print("COUNT DISTINCT Genre with Expression")
unique_genres_df_v2.show()

# Math aggregations

In [15]:
# min/max
max_rating_df = movies_df \
  .select(
    max(col("IMDB_Rating")).alias("max value of IMDB Rating"),
    min(col("IMDB_Rating")).alias("min value of IMDB Rating")
  )
max_rating_df.show()

In [16]:
max_rating_df_v2 = movies_df \
  .selectExpr(
    "max(IMDB_Rating) max_rating",
    "min(IMDB_Rating) as min_rating"
  )
max_rating_df_v2.show()

In [17]:
# сумма значений в столбце
us_industry_total_df = movies_df \
  .select(
    sum(col("US_Gross")) \
      .alias("Sum US Gross")
  )
us_industry_total_df.show()

In [18]:
us_industry_total_df_v2 = movies_df.selectExpr("sum(US_Gross) as Sum_US_Gross")
us_industry_total_df_v2.show()

In [19]:
# avg
avg_rt_rating_df = movies_df.select(avg(col("Rotten_Tomatoes_Rating")).alias("Average Rotten Tomatoes Rating"))
avg_rt_rating_df.show()

In [20]:
# mean/standard dev
rt_stats_df = movies_df.agg(
    mean(col("Rotten_Tomatoes_Rating")) \
      .alias("Mean of Rotten Tomatoes Rating"),
    stddev(col("Rotten_Tomatoes_Rating")) \
      .alias("Standard Deviation in Rotten Tomatoes Rating")
)
rt_stats_df.show()

# Grouping

In [21]:
# Группировка по колонке. NULL значения так же образуют группу
count_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    count()

count_by_genre_df.show()

In [22]:
count_by_genre_df.explain()

In [23]:
# Операции над группами
avg_rating_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    avg("IMDB_Rating")

avg_rating_by_genre_df.show()

In [24]:
avg_rating_by_genre_df.explain()

In [25]:
# Множетственные операции над группами
aggregations_by_genre_df = movies_df. \
  groupBy(col("Major_Genre")). \
  agg(
    # use strings here for column names
    count("*") \
      .alias("N Movies"),
    avg("IMDB_Rating") \
      .alias("Average Rating")
  )

aggregations_by_genre_df.show()

In [26]:
aggregations_by_genre_df.explain()

# Sorting

In [27]:
# Числа, строки и даты можно сортировать
best_movies_df = movies_df \
  .selectExpr("IMDB_Rating", "Title") \
  .orderBy(col("IMDB_Rating").desc())
best_movies_df.show(10, False)

In [28]:
# Сортировка с NULLS_FIRST
proper_worst_movies_df = movies_df \
  .selectExpr("IMDB_Rating", "Title") \
  .orderBy(col("IMDB_Rating").desc_nulls_first())

proper_worst_movies_df.show(10, False)

# Упражнения

1. Получить общую кассу всех фильмов (сколько фильмы заработали все вместе)
1. Посчитать общее колличество режиссеров (один режиссер мог снять несколько фильмов)
1. Вычислить среднее (mean) и стандартное отклонение (stddev) для прибыли с американского проката (US_Gross)
1. Для каждого режиссера вычислить средний (avg) рейтинг на IMDB и среднюю (avg) прибыль в американском прокате 
1. Вычислить среднюю (avg) разность между IMDB и Rotten Tomatoes рейтингами

# Joins

In [29]:
guitars_df = spark.read.json("data/guitars")
guitar_players_df = spark.read.json("data/guitarPlayers")
bands_df = spark.read.json("data/bands")
guitars_df.printSchema()
guitar_players_df.printSchema()
bands_df.printSchema()

In [30]:
# INNER JOIN: все строки, которые есть и в "правом", и в "левом" множестве, для которых верно условие
join_condition = guitar_players_df.band == bands_df.id

guitarists_bands_df = guitar_players_df \
  .join(
    bands_df,
    join_condition,
    # "inner" # <-- 'inner' == Default
  )

guitarists_bands_df.show()

In [31]:
guitarists_bands_df.explain()

In [55]:
# Привести колонку "name" к верхнему регистру
# Обратите внимание, что `guitarists_bands_df` строится на базе `bands_df`, поэтому можно использовать его колонки
guitarists_bands_upper_df = guitarists_bands_df.select(upper(bands_df.name))
guitarists_bands_upper_df.show()

In [56]:
guitarists_bands_upper_df.explain()

In [58]:
# left outer = все строки с лева, для которых условие верно. Если строки с таким ключом нет в правом множестве, то поставить null
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_outer"
  ) \
  .show()

In [59]:
# Right outer = все строки с права, для которых условие верно. Если строки с таким ключом нет в левом множестве, то поставить null
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "right_outer"
  ) \
  .show()

In [60]:
# full outer join = все строки справа и с лева, для которых верно условие, поставить NULL если строки с соответствующим ключом нет в правом или в левом множестве
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "outer"
  ) \
  .show()

In [61]:
# Соединить по одной колонке
guitar_players_df \
  .join(
    bands_df,
    "id"
  ) \
  .show()

In [30]:
# left semi joins = все элементы в левом множестве, для которых есть соответствующий элемент в правом множестве для которого верно условие
# Похоже на фильтр
# Эквивалентный SQL (колонок из правого множества нет): select * from guitar_players WHERE EXISTS (...)
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_semi"
  ) \
  .show()

In [62]:
# Исключить из левого множества каждую строку, для ключа которой есть строка с таким ключом в правом множестве
# Похоже на разность множеств
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_anti"
  ) \
  .show()

In [63]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://postgres:5432/spark"
user = "docker"
password = "docker"

In [64]:
def read_table(table_name):
    return spark.read. \
        format("jdbc"). \
        option("driver", driver). \
        option("url", url). \
        option("user", user). \
        option("password", password). \
        option("dbtable", "public." + table_name). \
        load()

employees_df = read_table("employees")
salaries_df = read_table("salaries")
dept_managers_df = read_table("dept_manager")
dept_emp_df = read_table("dept_emp")
departments_df = read_table("departments")

In [65]:
all_df = employees_df \
  .join(dept_emp_df, "emp_no") \
  .join(F.broadcast(departments_df), "dept_no") \
  .join(salaries_df, "emp_no")

all_df.show(10)
all_df.explain()

# Задания

Загрузить следующие таблицы из базы данных Postgres: `employees`, `salaries`, `dept_emp`

1. Получить список всех сотрудников и их максимальные зарплаты
1. Получить список всех сотрудников, кто никогда не был менеджером
1. Для каждого сотрудника, найти разницу между их зарплатой (текущей/последней) и максимальной зарплатой в их отделе